## Borz0 1.2.0

## Initializing :

In [ ]:
#Importing the os, Spark and other Complementary Libraries in Python 3.12.7, then Checking the Version of Spark :
import os
import csv
from IPython.display import clear_output
import pyspark
import matplotlib.pyplot as Moth
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc
print("PySpark version :", pyspark.__version__)

##  Starting Borz0 1.2.0 :

In [ ]:
#Build the ETL PipeLine
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11"

spark = SparkSession.builder \
    .appName("Simple Test") \
    .master("local[*]") \
    .getOrCreate()

print("Spark session created successfully!")

## Load Data :

In [ ]:
#Ask User for the Data they want to use, and then load it after checking it exists:
while True:
    data_loaded = input("Please enter the Data you want to Load : ")
    if os.path.isfile(data_loaded):
        print(f"{data_loaded} exists in the current directory and will be loaded")
        break
    else:
        print(f"Fatal Error, {data_loaded} does not exist in the directory")

## Visualizing the Data

In [ ]:
#Visualizing the Data
file_path = data_loaded
howMany_shown = int(input("Please enter how many rows you want to display"))
df_spark = spark.read.csv(file_path, header=True, inferSchema=True)
df_spark.show(howMany_shown)

## Identifying the Parameters

In [ ]:
df_spark.printSchema()

## Searching via a parameter, a certain Data || Find Duplicates

In [ ]:
#Search Algorythm, looking for a certain data like "SELECT * FROM clients WHERE name = Papatate"

# Load the CSV into PySpark DataFrame
df = spark.read.csv(data_loaded, header=True, inferSchema=True)

#Asking the User about the Precise Data he wants to search within a certain Paramter
f_Param = input("Please enter The Parameter you want to sear in :")
f_data = input(f"Please Enter the data you want to search with the {f_Param} parameter")

#Initializing a GLOBAL Variable to treat the loaded Data like a SQL DataBase
df.createOrReplaceTempView("GLOBAL")

#Try to Display the precise Data We're Looking for, as you can see, it also works to find Duplicates
try:
    query = f"SELECT * FROM GLOBAL WHERE `{f_Param}` = '{f_data}'"
    spark.sql(query).show()
except Exception as e: #saves the exception parameter in e
    print(f"Error executing query: {e}")

## Ordering Data with Parameter :

In [ ]:
#Order Data With a parametter

#First, check if the parameter exists in the current Database
while True:
    parameter = input("Please enter the Parameter to use for Sorting the Data")
    clear_output(wait=True)
    if parameter in df_spark.columns:
        break
    else:
        print("Error, this parameter doesnt exist in the current csv file")

#Asking the User how many rows they want to display
howMany_shown = int(input("Please enter how many rows you want to display"))

#Choosing if the user wants to use desc or asc to determine the order of the Data used as a point of reference
while True:
    param2 = input("Do you want to start with the asc (type = asc) or the biggest (type = desc)")
    clear_output(wait=True)
    if param2 == "asc":
        sorted_df = df_spark.orderBy(parameter)
        break
    elif param2 == "desc":
        sorted_df = df_spark.orderBy(desc(parameter))
        break
    else:
        print("Error, please enter desc or asc")

#indicating which parameter that has been used to clarify the graph
print(f"Data sorted by using {parameter} as the reference")
sorted_df.show(howMany_shown)

## Create a folder and Store the Data into it :

In [ ]:
#Creating the file and naming it
fldr_name = input("Please enter a name for the Folder")
sorted_df.coalesce(1).write.csv(fldr_name, header=False, mode='overwrite')


## Stopping Borz0 :

In [ ]:
#Stopping the ETL Pipeline to release memory and ressources
spark.stop()
#Checking if Spark is stoppped
try:
    if spark.sparkContext._jsc is None:
        print("Spark successfully stopped")
    else:
        print("Error: Spark is not stopped")
#if the upper part did not work :
except AttributeError:
    print("Spark successfully stopped")